In [2]:
import wfdb
import numpy as np
import pandas as pd

def get_annotations(record_path):
    """
    Reads annotations from a PhysioNet record, extracts relevant symbols, 
    and returns a DataFrame in the specified format.

    Args:
        record_path (str): Path to the PhysioNet record (e.g., 'vfdb/100')

    Returns:
        pandas.DataFrame: DataFrame with columns 'Time', 'Sample', 'Type', 'Sub', 'Chan', 'Num', 'Aux'
    """

    annotation = wfdb.rdann(record_path, 'atr')

    df = pd.DataFrame({
        'Time': np.round(annotation.sample / 250, 3),  # Assuming sampling frequency of 250 Hz
        'Sample': annotation.sample,
        #'Type': annotation.symbol,
        #'Sub': annotation.subtype,
        #'Chan': annotation.chan,
        #'Num': annotation.num,
        'Aux': annotation.aux_note
    })

    # Replace '+' with an empty string
    #df['Type'].replace('+', '', inplace=True)

    # Convert Time to Time Elapsed format
    df['Time'] = np.round(annotation.sample / 250, 3)  


    return df


record_path =  r"C:\Users\Subham\Capstone\VFDB_pyhton\418"  # Use raw string or double backslashes

annotations_df = get_annotations(record_path)



# Add serial number column
annotations_df.insert(0, 'Serial Number', range(1, len(annotations_df) + 1)) 

# Export to CSV
annotations_df.to_csv('ann_418.csv', index=False) 

In [12]:
import wfdb
import numpy as np
import pandas as pd
import os

# Constants
SEGMENT_DURATION = 5  # Seconds
SAMPLING_FREQUENCY = 250  # Hz
SIGNAL_DURATION = 35 * 60  # 35 minutes in seconds
BASE_INPUT_DIR = "C:\\Users\\Subham\\Capstone\\VFDB_pyhton"
OUTPUT_DIR = "C:\\Users\\Subham\\Capstone\\VFDB_pyhton\\Annotation files"

def get_annotations(record_path):
    """
    Reads annotations from a PhysioNet record, extracts relevant symbols, 
    and returns a DataFrame in the specified format.

    Args:
        record_path (str): Path to the PhysioNet record (e.g., 'vfdb/100')

    Returns:
        pandas.DataFrame: DataFrame with columns 'Time', 'Sample', 'Aux'
    """

    annotation = wfdb.rdann(record_path, 'atr')

    df = pd.DataFrame({
        'Time': np.round(annotation.sample / 250, 3),  # Assuming sampling frequency of 250 Hz
        'Sample': annotation.sample,
        'Aux': annotation.aux_note
    })

    return df

def process_record(file_number):
    record_name = str(file_number)  
    record_path = os.path.join(BASE_INPUT_DIR, record_name) 
    annotations_df = get_annotations(record_path)

    # Segment the data
    start_time = 0
    segment_num = 1
    current_annotation = None
    output_data = []

    while start_time < SIGNAL_DURATION:
        end_time = start_time + SEGMENT_DURATION
        segment_df = annotations_df[(annotations_df['Time'] >= start_time) & (annotations_df['Time'] < end_time)]

        for _, row in segment_df.iterrows():
            if row['Aux'].startswith('('):
                current_annotation = row['Aux'][1:] 

        if end_time > SIGNAL_DURATION:
            end_time = SIGNAL_DURATION

        output_data.append({
            'Segment': segment_num,
            'Start Time': start_time,
            'End Time': end_time,
            'Active Annotations': current_annotation or "None"
        })

        start_time = end_time
        segment_num += 1

    # Export to CSV
    output_df = pd.DataFrame(output_data) 
    output_filename = os.path.join(OUTPUT_DIR, 'ann_' + record_name + '.csv')
    output_df.to_csv(output_filename, index=False)

# Main Processing Loop
for file_number in range(614, 616):  
    process_record(file_number)
